# Huggingface library tutorial

* HuggingFace
  * Transformer 라이브러리를 구축하고 유지하는 회사

* HuggingFace library
  * 공식화된 라이브러리
  * language model은 huggingface의 transformers 를 기반으로 만들어짐

* 모델 다운로드 및 작업 시작
  * 아래 3줄만 있으면 됨
  * 모델에 필요한 parameter나 configuration, vocab file 등을 다운로두함

In [ ]:
from transformers import TFAutoModel, AutoTokenizer
model = TFAutoModel.from_pretrained("<model-name>")
tokenizer = AutoTokenizer.from_pretrained("<model-name>")

* 모델 검색
  * HuggingFace 사이트의 검색창을 이용하여 모델 검색

### Tokenizer 실습
* transformers 사용

In [ ]:
# !pip install transformers

## Tokenizer 응용
* BERT model과 BERT tokenizer가 필요함



In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer

* 실습에서 사용할 모델 : Transformer기반의 대표 모델인 multi-lingual bert model
  * 한국어도 포함되어 있음

In [ ]:
# Store the model we want to use
MODEL_NAME = "bert-base-multilingual-cased"

# We need to create the model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

* multi-lingual model이기 때문에 wordpiece vocab이 12만개정도 됨
  * 그 중 한국어는 8천개 정도

In [ ]:
print(tokenizer.vocab_size)

* 한국어 corpus를 이용해서 vocab 을 만들 때, 3만개 정도로 정의하면 적절함
  * 한자와 특수기호도 대부분 들어감

In [ ]:
for i, key in enumerate(tokenizer.get_vocab()):
    print(key)
    if i > 50:
        break

In [ ]:
text = "이순신은 조선 중기의 무신이다."

* Multi-lingual bert model은 `BERTTokenizeFast` class로 되어 있음

In [ ]:
print(type(tokenizer))

* tokenizer 테스트
  * `return_tensors="pt"` : pyTorch로 return
  * tokenize된 정보들이 dictionary화 되어 `tokenized_input_text` 에 들어가있음
  * 'input_ids' : input text를 tokenizing 한 후의 vocab id
    * 이 vocab id가 모델에 input으로 입력됨
  * 'token_type_ids' : segment id
    * 입력된 sentence index 번호(?)
    * 같은 문장의 token들은 동일한 id를 가짐
  * 'attention_mask' : 
    * padding은 0으로 초기화됨

In [ ]:
tokenized_input_text = tokenizer(text, return_tensors="pt")
for key, value in tokenized_input_text.items():
    print("{}:\n\t{}".format(key, value))

In [ ]:
print(tokenized_input_text['input_ids'])    # input text를 tokenizing한 후 vocab의 id
print(tokenized_input_text.input_ids)
print(tokenized_input_text['token_type_ids'])   # segment id (sentA or sentB)
print(tokenized_input_text.token_type_ids)
print(tokenized_input_text['attention_mask'])   # special token (pad, cls, sep) or not
print(tokenized_input_text.attention_mask)

* `tokenizer.tokenize()` : 입력된 input을 tokenizing함
    * 명시적으로 보여줌
    * tokenizer의 default값이 자동으로 함수에 special token을 부착하도록 되어 있음
      * BERT의 입력은 항상 [CLS]가 문장 앞에, [SEP]가 문장 끝에 붙게 됨

* encoding & decoding
  

In [ ]:
  tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)
input_ids = tokenizer.encode(text)
print(input_ids)
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)

* special token을 추가하고 싶지 않을 경우 옵션 변경
  * `add_special_tokens=False` : [CLS], [SEP]가 붙지 않음

In [ ]:
tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print(tokenized_text)
input_ids = tokenizer.encode(text, add_special_tokens=False)
print(input_ids)
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)

* tokenizing할 때, padding과 truncation이 중요함
  * `add_special_tokens=False`, `max_length=5`, `truncation=True` 이면 [CLS]와 [SEP]가 붙지 않은 상태의 token들(입력문장을 tokenizing했을 때의 token들) 중에 5개의 token만 남기고 잘라냄

* tokenizer token 단위로 자르는 것이기 때문에 원래의 input 문장의 length와 관련이 없음

In [ ]:
tokenized_text = tokenizer.tokenize(
    text,
    add_special_tokens=False,
    max_length=5,
    truncation=True
    )
print(tokenized_text)
# ['이', '##순', '##신', '##은', '조선']

input_ids = tokenizer.encode(
    text,
    add_special_tokens=False,
    max_length=5,
    truncation=True
    )
print(input_ids)
# [9638, 119064, 25387, 10892, 59906]
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
# 이순신은 조선

* padding
  * tokenizer 함수에 padding옵션을 넣어주면 됨
    * 원하는 padding조건에 따라 옵션이 바뀜
  * 'max_length' : 기본 옵션
    * max length만큼 padding을 뒤에 부착해서 채움

In [ ]:
print(tokenizer.pad_token) # [PAD]
print(tokenizer.pad_token_id) # 0

tokenized_text = tokenizer.tokenize(
    text,
    add_special_tokens=False,
    max_length=20,
    padding="max_length"
    )
print(tokenized_text)
# ['이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '.', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']

input_ids = tokenizer.encode(
    text,
    add_special_tokens=False,
    max_length=20,
    padding="max_length"
    )
print(input_ids)
# [9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119, 0, 0, 0, 0, 0, 0, 0, 0, 0]

decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
# 이순신은 조선 중기의 무신이다. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

* 새로운 token 추가

In [ ]:
print(tokenizer.vocab_size) # 119547

* 학습되어있는 정상적인 문장이 아니기 때문에 tokenizing이 되지 않음
  * [UNK]로 바뀜
  * [UNK]가 많을수록 원본 문장의 의미가 희석됨

In [ ]:
text = "깟뻬뜨랑 리뿔이 뜨럽거 므리커럭이 케쇽 냐왜쇼 우뤼갸 쳥쇼섀료다혀뚜여"

tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print(tokenized_text)
# ['[UNK]', '리', '##뿔', '##이', '뜨', '##럽', '##거', '므', '##리', '##커', '##럭', '##이', '[UNK]', '냐', '##왜', '##쇼', '[UNK]', '[UNK]']
input_ids = tokenizer.encode(text, add_special_tokens=False)
print(input_ids)
# [100, 9238, 119021, 10739, 9151, 118867, 41521, 9308, 12692, 106826, 118864, 10739, 100, 9002, 119164, 119060, 100, 100]
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
# [UNK] 리뿔이 뜨럽거 므리커럭이 [UNK] 냐왜쇼 [UNK] [UNK]

* 원하는 task, 작업할 focus에 맞춰서 tokenizer 수정이 필요함

* 임의로 [UNK]에 해당하는 vocab들을 `add_tokens()`를 사용하여 token에 추가함
  * list형태로 입력
  * tokenizing할 때 이 token들이 반영됨

In [ ]:
added_token_num = tokenizer.add_tokens(["깟뻬뜨랑", "케쇽", "우뤼갸", "쳥쇼", "섀료"])
print(added_token_num)
# 5

tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print(tokenized_text)
# ['깟뻬뜨랑', '리', '##뿔', '##이', '뜨', '##럽', '##거', '므', '##리', '##커', '##럭', '##이', '케쇽', '냐', '##왜', '##쇼', '우뤼갸', '쳥쇼', '섀료', '다', '##혀', '##뚜', '##여']
input_ids = tokenizer.encode(text, add_special_tokens=False)
print(input_ids)
# [119547, 9238, 119021, 10739, 9151, 118867, 41521, 9308, 12692, 106826, 118864, 10739, 119548, 9002, 119164, 119060, 119549, 119550, 119551, 9056, 80579, 118841, 29935]
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
# 깟뻬뜨랑 리뿔이 뜨럽거 므리커럭이 케쇽 냐왜쇼 우뤼갸 쳥쇼 섀료 다혀뚜여

In [ ]:
print(tokenizer.vocab_size) # 119547

 * 특정 역할을 위한 special token도 추가할 수 있음
  * ex. [ENTITY]
  * 문장에만 추가하면 token으로 인식함

In [ ]:
text = "[SHKIM]이순신은 조선 중기의 무신이다.[/SHKIM]"
# [ENTITY]이순신[/ENTITY]
tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print(tokenized_text)
# ['[', 'SH', '##KI', '##M', ']', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '.', '[', '/', 'SH', '##KI', '##M', ']']
input_ids = tokenizer.encode(text, add_special_tokens=False)
print(input_ids)
# [164, 38702, 59879, 11517, 166, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119, 164, 120, 38702, 59879, 11517, 166]
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
# [ SHKIM ] 이순신은 조선 중기의 무신이다. [ / SHKIM ]

* `add_special_tokens()` 함수를 사용하여 special token을 추가
  * dictionary 형태로 입력

In [ ]:
text = "[SHKIM]이순신은 조선 중기의 무신이다.[/SHKIM]"

added_token_num += tokenizer.add_special_tokens({"additional_special_tokens":["[SHKIM]", "[/SHKIM]"]})
tokenized_text = tokenizer.tokenize(text, add_special_tokens=False)
print(tokenized_text)
# ['[SHKIM]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '.', '[/SHKIM]']
input_ids = tokenizer.encode(text, add_special_tokens=False)
print(input_ids)
# [119552, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119, 119553]
decoded_ids = tokenizer.decode(input_ids)
print(decoded_ids)
# [SHKIM] 이순신은 조선 중기의 무신이다. [/SHKIM]
decoded_ids = tokenizer.decode(input_ids,skip_special_tokens=True)
print(decoded_ids)
# 이순신은 조선 중기의 무신이다.

* `add_token_num` : model을 resize하는 기준이 됨
  * prefix된 model은 embedding되어있는 vocab size가 고정되어 있음
  * vocab size를 벗어나는 id가 input으로 들어오면 에러 발생함
  * model을 resize해주어야함

In [ ]:
print(added_token_num) # 7

* 자연어처리 task

In [ ]:
# Single segment input
single_seg_input = tokenizer("이순신은 조선 중기의 무신이다.")

# Multiple segment input
multi_seg_input = tokenizer("이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다.")

print("Single segment token (str): {}".format(tokenizer.convert_ids_to_tokens(single_seg_input['input_ids'])))
# Single segment token (str): ['[CLS]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '.', '[SEP]']
print("Single segment token (int): {}".format(single_seg_input['input_ids']))
# Single segment token (int): [101, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119, 102]
print("Single segment type       : {}".format(single_seg_input['token_type_ids']))
# Single segment type       : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Segments are concatened in the input to the model, with 
print()
print("Multi segment token (str): {}".format(tokenizer.convert_ids_to_tokens(multi_seg_input['input_ids'])))
# Multi segment token (str): ['[CLS]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '.', '[SEP]', '그는', '임', '##진', '##왜', '##란', '##을', '승', '##리로', '이', '##끌', '##었다', '.', '[SEP]']
print("Multi segment token (int): {}".format(multi_seg_input['input_ids']))
# Multi segment token (int): [101, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119, 102, 17889, 9644, 18623, 119164, 49919, 10622, 9484, 100434, 9638, 118705, 17706, 119, 102]
print("Multi segment type       : {}".format(multi_seg_input['token_type_ids']))
# Multi segment type       : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

* list형태로 입력하면 list 형태로 출력이 됨

In [ ]:
# Padding highlight
tokens = tokenizer(
    ["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."], 
    padding=True  # First sentence will have some PADDED tokens to match second sequence length
)

for i in range(2):
    print("Tokens (int)      : {}".format(tokens['input_ids'][i]))
    print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in tokens['input_ids'][i]]))
    print("Tokens (attn_mask): {}".format(tokens['attention_mask'][i]))
    print()

'''
Tokens (int)      : [101, 9638, 119064, 25387, 10892, 59906, 9694, 46874, 9294, 25387, 11925, 119, 102, 0]
Tokens (str)      : ['[CLS]', '이', '##순', '##신', '##은', '조선', '중', '##기의', '무', '##신', '##이다', '.', '[SEP]', '[PAD]']
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]

Tokens (int)      : [101, 17889, 9644, 18623, 119164, 49919, 10622, 9484, 100434, 9638, 118705, 17706, 119, 102]
Tokens (str)      : ['[CLS]', '그는', '임', '##진', '##왜', '##란', '##을', '승', '##리로', '이', '##끌', '##었다', '.', '[SEP]']
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
'''

### BERT 모델 테스트

* BERT 모델을 이용하여 [MASK] token을 예측

In [ ]:
text = "이순신은 [MASK] 중기의 무신이다."
tokenized_text = tokenizer.tokenize(text)

print(tokenized_text)
# ['이', '##순', '##신', '##은', '[MASK]', '중', '##기의', '무', '##신', '##이다', '.']

* `pipeline`
  * mask를 채운 결과를 return

In [ ]:
from transformers import pipeline

nlp_fill = pipeline('fill-mask', model=MODEL_NAME)
nlp_fill("이순신은 [MASK] 중기의 무신이다.")

'''
/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[{'score': 0.8747126460075378,
  'sequence': '이순신은 조선 중기의 무신이다.',
  'token': 59906,
  'token_str': '조선'},
 {'score': 0.06436426192522049,
  'sequence': '이순신은 청 중기의 무신이다.',
  'token': 9751,
  'token_str': '청'},
 {'score': 0.010954886674880981,
  'sequence': '이순신은 전 중기의 무신이다.',
  'token': 9665,
  'token_str': '전'},
 {'score': 0.0046471720561385155,
  'sequence': '이순신은종 중기의 무신이다.',
  'token': 22200,
  'token_str': '##종'},
 {'score': 0.0036106714978814125,
  'sequence': '이순신은기 중기의 무신이다.',
  'token': 12310,
  'token_str': '##기'}]
'''

* 출력 결과 확인
  * tokenizer의 결과를 model의 input으로 주어 결과를 출력함
  * Token wise output : 
    * 입력된 문장은 13개의 token으로 이루어져 있음
    * 13개의 token에 대한 vector값(768)
  * Pooled output : 
    * CLS token의 vector를 얻어냄
    * 문장 input(1)과 768차원의 문장 vector output

In [ ]:
tokens_pt = tokenizer("이순신은 조선 중기의 무신이다.", return_tensors="pt")
for key, value in tokens_pt.items():
    print("{}:\n\t{}".format(key, value))

'''
input_ids:
	tensor([[   101,   9638, 119064,  25387,  10892,  59906,   9694,  46874,   9294,
          25387,  11925,    119,    102]])
token_type_ids:
	tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
'''

outputs = model(**tokens_pt)
last_hidden_state = outputs.last_hidden_state
pooler_output = outputs.pooler_output

print("\nToken wise output: {}, Pooled output: {}".format(last_hidden_state.shape, pooler_output.shape))
# Token wise output: torch.Size([1, 13, 768]), Pooled output: torch.Size([1, 768])

In [ ]:
print(pooler_output) # [CLS] token to 768 dimension

* vocab을 추가한 경우 반드시 model의 embedding layer 사이즈를 늘려야 함

In [ ]:
print(model.get_input_embeddings())
# Embedding(119547, 768, padding_idx=0)
model.resize_token_embeddings(tokenizer.vocab_size + added_token_num)
print(model.get_input_embeddings())
# Embedding(119554, 768)

* [CLS] token을 활용하여 문장의 유사도 측정
  * pooled output은 문장 input과 문장 vector output을 출력함
  * 두 문장간의 유사도 계산이 가능함

In [ ]:
sent1 = tokenizer("오늘 하루 어떻게 보냈나요?", return_tensors="pt")
sent2 = tokenizer("오늘은 어떤 하루를 보내셨나요?", return_tensors="pt")
sent3 = tokenizer("이순신은 조선 중기의 무신이다.", return_tensors="pt")
sent4 = tokenizer("깟뻬뜨랑 리뿔이 뜨럽거 므리커럭이 케쇽 냐왜쇼 우뤼갸 쳥쇼섀료다혀뚜여", return_tensors="pt")


outputs = model(**sent1)
sent_1_pooler_output = outputs.pooler_output

outputs = model(**sent2)
sent_2_pooler_output = outputs.pooler_output

outputs = model(**sent3)
sent_3_pooler_output = outputs.pooler_output

outputs = model(**sent4)
sent_4_pooler_output = outputs.pooler_output

* `nn.CosineSimilarity()` 함수를 사용하여 유사도를 측정함
  * 유사도가 높다고 해도 의미가 유사하지 않을 수 있음

In [ ]:
from torch import nn

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
print(cos(sent_1_pooler_output, sent_2_pooler_output))
# tensor([0.9757], grad_fn=<DivBackward0>) # 유사도가 높음
print(cos(sent_2_pooler_output, sent_3_pooler_output))
# tensor([0.6075], grad_fn=<DivBackward0>)
print(cos(sent_3_pooler_output, sent_4_pooler_output))
# tensor([0.6167], grad_fn=<DivBackward0>)
print(cos(sent_1_pooler_output, sent_4_pooler_output))
# tensor([0.9389], grad_fn=<DivBackward0>) # 유사도가 높음